<a href="https://colab.research.google.com/github/41371109h/114-1/blob/main/hw4_%E6%96%87%E5%AD%97%E8%B3%87%E6%96%99%E5%B0%8F%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
# 安裝所有必要的 Python 套件
!pip -q install gspread gspread_dataframe google-auth google-auth-oauthlib google-auth-httplib2 \
               gradio pandas beautifulsoup4 google-generativeai python-dateutil scikit-learn jieba

# 確保 jieba 字典準備就緒 (非必要，但有助於確保第一次運行成功)
import jieba
try:
    jieba.lcut("初始化")
except Exception:
    pass

In [29]:
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup
import google.generativeai as genai
from google.colab import auth, userdata
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth import default
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba
from collections import Counter, defaultdict
from itertools import tee

# --- Google Colab 認證 (允許存取 Sheets) ---
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# --- Gemini API 設定 ---
try:
    # 從 Colab Secrets 中獲取 API 金鑰
    api_key = userdata.get('wt')
    genai.configure(api_key=api_key)
    # 使用 2.5 Flash 進行快速洞察生成
    GEMINI_MODEL = genai.GenerativeModel('gemini-2.5-flash')
except Exception as e:
    print(f"⚠️ Gemini API Key 設置失敗或無法讀取：{e}")
    GEMINI_MODEL = None


# --- 常數定義 ---
SHEET_URL = "https://docs.google.com/spreadsheets/d/1eqVMllof4j4Xxvmnzny7z8ojKifHj-CS3a1IUJ2brfg/edit?gid=0#gid=0"
WORKSHEET_NAME = "PTT電影分析"
TIMEZONE = "Asia/Taipei"

PTT_HEADER = [
    "post_id","title","url","date","author","nrec","created_at",
    "fetched_at","content"
]
TERMS_HEADER = ["term","freq","df_count","tfidf_mean","examples"]

PTT_MOVIE_INDEX = "https://www.ptt.cc/bbs/movie/index.html"
PTT_COOKIES = {"over18": "1"}


In [30]:
# --- Google Sheet 輔助函式 ---
def ensure_spreadsheet(name):
    """確保試算表存在，若無則創建"""
    try:
        sh = gc.open(name)
    except gspread.SpreadsheetNotFound:
        sh = gc.create(name)
    return sh

def ensure_worksheet(sh, title, header):
    """確保工作表存在，並檢查/設定正確的表頭"""
    try:
        ws = sh.worksheet(title)
    except gspread.WorksheetNotFound:
        ws = sh.add_worksheet(title=title, rows="1000", cols=str(len(header)+5))
        ws.update([header])
    # 檢查並修正表頭
    data = ws.get_all_values()
    if not data or (data and data[0] != header):
        ws.clear()
        ws.update([header])
    return ws

def tznow():
    """回傳帶有時區的時間戳記"""
    return dt.now(gettz(TIMEZONE))

def read_df(ws, header):
    """從 Worksheet 讀取資料為 DataFrame，並處理空值與型別"""
    df = get_as_dataframe(ws, evaluate_formulas=True, header=0)
    if df is None or df.empty:
        return pd.DataFrame(columns=header)
    df = df.fillna("")
    for c in header:
        if c not in df.columns:
            df[c] = ""
    # 確保數值欄位為數字（此處我們只保留字串型態，讓 pandas 處理）
    return df[header]

def write_df(ws, df, header):
    """將 DataFrame 寫入 Worksheet"""
    if df.empty:
        ws.clear()
        ws.update([header])
        return
    # 轉字串避免 gspread 型別問題
    df_out = df.copy()
    for c in df_out.columns:
        df_out[c] = df_out[c].astype(str)
    ws.clear()
    ws.update([header] + df_out.values.tolist())

def read_ptt_posts_df():
    """讀取 PTT 貼文資料框"""
    return read_df(ws_ptt_posts, PTT_HEADER).copy()

def read_terms_df():
    """讀取關鍵詞資料框"""
    return read_df(ws_ptt_terms, TERMS_HEADER).copy()


# --- 初始化工作表與 DataFrame ---
sh = ensure_spreadsheet(WORKSHEET_NAME)
ws_ptt_posts = ensure_worksheet(sh, "ptt_movie_posts", PTT_HEADER)
ws_ptt_terms = ensure_worksheet(sh, "ptt_movie_terms", TERMS_HEADER)

# 全域 DataFrame 變數初始化 (用於程式內存操作)
ptt_posts_df = read_ptt_posts_df()
terms_df = read_terms_df()

In [31]:
# --- PTT 電影版爬蟲函式群 ---
def _get_soup(url):
    """發送請求並獲取 BeautifulSoup 物件"""
    r = requests.get(url, timeout=15, headers={"User-Agent":"Mozilla/5.0"}, cookies=PTT_COOKIES)
    r.raise_for_status()
    return BeautifulSoup(r.text, "html.parser")

def _get_prev_index_url(soup):
    """獲取上一頁的 URL"""
    btns = soup.select("div.btn-group-paging a.btn.wide")
    for a in btns:
        if "上頁" in a.get_text(strip=True):
            href = a.get("href")
            if href:
                return "https://www.ptt.cc" + href
    return None

def _parse_nrec(nrec_span):
    """解析推文數 (爆/X/數字)"""
    if not nrec_span: return 0
    txt = nrec_span.get_text(strip=True)
    if txt == "爆": return 100
    if txt.startswith("X"):
        try: return -int(txt[1:])
        except: return -10
    try: return int(txt)
    except: return 0

def _extract_post_list(soup):
    """從索引頁面提取文章列表"""
    posts = []
    for r in soup.select("div.r-ent"):
        a = r.select_one("div.title a")
        if not a: continue
        title = a.get_text(strip=True)
        url = "https://www.ptt.cc" + a.get("href")
        author = r.select_one("div.author").get_text(strip=True)
        date = r.select_one("div.date").get_text(strip=True)
        nrec = _parse_nrec(r.select_one("div.nrec span"))
        posts.append({
            "title": title, "url": url, "author": author, "date": date, "nrec": nrec
        })
    return posts

def _clean_ptt_content(soup):
    """清除內文中的推文區和 meta 資訊，提取乾淨的內文"""
    for p in soup.select("div.push"): p.decompose()
    main = soup.select_one("#main-content")
    if not main: return "", ""
    metas = main.select("div.article-metaline, div.article-metaline-right")
    for m in metas: m.decompose()
    text = main.get_text("\n", strip=True)
    if "--" in text:
        text = text.split("--")[0].strip()
    title_tag = soup.select_one("span.article-meta-value")
    meta_title = title_tag.get_text(strip=True) if title_tag else ""
    return text, meta_title

def crawl_ptt_movie(index_pages=3, min_push=0, keyword=""):
    """從最新 index.html 往前翻頁抓取文章，並寫入 Google Sheet"""
    global ptt_posts_df
    url = PTT_MOVIE_INDEX
    all_rows = []
    # 從當前 DataFrame 獲取已有的 URL 進行去重
    seen_urls = set(ptt_posts_df["url"].tolist()) if not ptt_posts_df.empty else set()

    start_time = time.time()

    for page_num in range(int(index_pages)):
        try:
            soup = _get_soup(url)
            posts = _extract_post_list(soup)
        except Exception as e:
            print(f"爬取第 {page_num+1} 頁失敗：{e}")
            break

        for p in posts:
            if p["nrec"] < int(min_push): continue
            if keyword and (keyword not in p["title"]): continue
            if p["url"] in seen_urls: continue

            # 抓正文
            try:
                art_soup = _get_soup(p["url"])
                content, meta_title = _clean_ptt_content(art_soup)
            except Exception:
                content, meta_title = "", ""

            final_title = p["title"] if p["title"] else (meta_title or "（無標題）")

            all_rows.append({
                "post_id": str(uuid.uuid4())[:8],
                "title": final_title[:200],
                "url": p["url"],
                "date": p["date"],
                "author": p["author"],
                "nrec": str(p["nrec"]),
                "created_at": tznow().isoformat(),
                "fetched_at": tznow().isoformat(),
                "content": content
            })

        prev = _get_prev_index_url(soup)
        if not prev: break
        url = prev
        time.sleep(0.5) # 避免被鎖

    if all_rows:
        new_df = pd.DataFrame(all_rows, columns=PTT_HEADER)
        ptt_posts_df = pd.concat([ptt_posts_df, new_df], ignore_index=True)
        # 寫回 Google Sheet
        write_df(ws_ptt_posts, ptt_posts_df, PTT_HEADER)
        total_count = len(ptt_posts_df)
        return f"✅ 新增 {len(all_rows)} 篇文章至 Sheet，目前總數：{total_count} 篇", ptt_posts_df
    else:
        total_count = len(ptt_posts_df)
        return f"ℹ️ 沒有新文章符合條件或內容已存在。目前總數：{total_count} 篇", ptt_posts_df

# --- 文本分析與關鍵詞統計 ---
def _tokenize_zh(text):
    """Jieba 分詞，並過濾掉長度小於 2 的詞彙"""
    text = re.sub(r"[^\u4e00-\u9fffA-Za-z0-9]+", " ", text)
    if not jieba:
        return [t for t in text.split() if len(t) > 1]
    return [w.strip() for w in jieba.lcut(text) if len(w.strip()) > 1]

def analyze_ptt_texts(topk=50, min_df=2):
    """執行 TF-IDF 關鍵詞分析，並將結果寫入 Google Sheet"""
    global ptt_posts_df, terms_df
    if ptt_posts_df.empty:
        terms_df = pd.DataFrame(columns=TERMS_HEADER)
        return "📭 尚無已抓取的文章，請先執行爬蟲。", terms_df, ""

    docs = []
    for _, r in ptt_posts_df.iterrows():
        # 將標題與內文拼起來作為文件
        docs.append((r["title"] or "") + " " + (r["content"] or ""))

    # --- 詞頻統計 ---
    freq = Counter()
    df_cnt = defaultdict(int)
    for doc in docs:
        toks = _tokenize_zh(doc)
        freq.update(toks)
        for t in set(toks):
            df_cnt[t] += 1

    # --- TF-IDF 計算 ---
    try:
        vec = TfidfVectorizer(tokenizer=_tokenize_zh, lowercase=False, min_df=int(min_df))
        X = vec.fit_transform(docs)
        terms = vec.get_feature_names_out()
        tfidf_mean = X.mean(axis=0).A1
        tfidf_map = dict(zip(terms, tfidf_mean))
    except Exception:
        tfidf_map = {t: 0.0 for t in freq.keys()}

    # --- 排序與 TopK 關鍵詞提取 ---
    candidates = [t for t in freq.keys() if t in tfidf_map]
    candidates.sort(key=lambda t: (round(tfidf_map.get(t,0.0), 6), freq[t]), reverse=True)
    top_terms = candidates[:int(topk)]

    # --- 範例句提取 ---
    examples = {}
    for term in top_terms:
        ex = ""
        for doc in docs:
            if term in doc:
                i = doc.find(term)
                s = max(0, i-15)
                e = min(len(doc), i+len(term)+15)
                ex = doc[s:e].replace("\n"," ")
                break
        examples[term] = ex

    # --- 輸出 DataFrame ---
    rows = []
    for t in top_terms:
        rows.append({
            "term": t,
            "freq": str(freq[t]),
            "df_count": str(df_cnt[t]),
            "tfidf_mean": f"{tfidf_map.get(t,0.0):.6f}",
            "examples": examples.get(t, "")
        })
    terms_df = pd.DataFrame(rows, columns=TERMS_HEADER)

    # 寫回 Google Sheet
    write_df(ws_ptt_terms, terms_df, TERMS_HEADER)
    msg_status = f"✅ 已完成文本分析，共 {len(docs)} 篇文章；已回寫 Top {len(top_terms)} 關鍵詞至 Sheet。"

    # 產生 Markdown 摘要給報告用
    md_lines = []
    md_lines.append(f"### 關鍵詞 Top {len(top_terms)}（依 TF-IDF 平均值優先，次序再以詞頻）")
    for i, t in enumerate(top_terms, 1):
        md_lines.append(f"{i}. **{t}** — tfidf≈{float(tfidf_map.get(t,0.0)):.4f}；freq={freq[t]}；df={df_cnt[t]}")

    return msg_status, terms_df, "\n".join(md_lines)

In [32]:
def generate_insights(df_terms, model):
    """
    串接 Gemini API，根據 Top 關鍵詞生成 5 句洞察摘要和 120 字結論。
    """
    if model is None or df_terms.empty:
        return "⚠️ Gemini API 未初始化或關鍵詞分析結果為空，無法生成 AI 洞察。"

    # 選擇前 10 個詞彙及其 TF-IDF 值
    top_terms_data = df_terms.head(10)[["term", "tfidf_mean", "freq", "examples"]].to_dict('records')

    # 構建給 Gemini 的指令
    system_prompt = (
        "你是一位專業的電影市場與輿情分析師。你的任務是根據輸入的 PTT 關鍵詞與統計數據，"
        "總結出對當前電影話題的洞察。請務必嚴格遵循以下格式要求："
        "1. **洞察摘要**: 輸出 5 個精確的句子，每個句子都必須以編號 1. 到 5. 開頭。"
        "2. **結論**: 輸出一段總長度約 120 字的總結段落。該結論必須在洞察摘要之後，並以標題 '## 分析結論' 開頭。"
        "請只回傳請求的 Markdown 內容，不要包含任何額外的前言或後記。"
    )

    user_query = f"請根據以下 PTT 電影版熱門關鍵詞（已按 TF-IDF 平均值排序）進行分析：\n\n{json.dumps(top_terms_data, ensure_ascii=False)}"

    try:
        response = model.generate_content(
            contents=[
                {"role": "user", "parts": [{"text": user_query}]}
            ],
            system_instruction=system_prompt,
        )
        return response.text
    except Exception as e:
        return f"⚠️ Gemini API 呼叫失敗：{e}"


def full_analysis_workflow(index_pages, min_push, keyword, topk, min_df):
    """
    一鍵執行完整的爬蟲、分析和 AI 洞察流程。
    """
    # 1. 爬蟲階段
    msg_crawl, df_posts = crawl_ptt_movie(index_pages, min_push, keyword)

    # 2. 文本分析階段 (從 Sheet 讀回最新數據)
    global ptt_posts_df
    ptt_posts_df = df_posts # 使用剛爬取/合併的最新 DF 進行分析
    msg_analysis, df_terms, md_analysis = analyze_ptt_texts(topk=topk, min_df=min_df)

    # 3. AI 洞察生成階段
    gemini_output = generate_insights(df_terms, GEMINI_MODEL)

    # 4. 組合所有結果
    final_report = f"""
# PTT 電影版輿情分析報告

## 流程狀態
- 爬蟲結果：{msg_crawl}
- 文本分析：{msg_analysis}

---

## 關鍵詞與詞頻統計
{md_analysis}

---

## 🤖 Gemini AI 洞察與總結

{gemini_output}
"""
    return final_report, df_terms

def _refresh_data():
    """重新從 Sheet 讀取最新數據到內存"""
    global ptt_posts_df, terms_df
    ptt_posts_df = read_ptt_posts_df()
    terms_df = read_terms_df()
    return ptt_posts_df, terms_df


In [33]:
# 載入最新的數據以供初始化顯示
_refresh_data()

with gr.Blocks(title="PTT 電影版 AI 輿情分析") as demo:
    gr.Markdown("# 🎬 PTT 電影版 AI 輿情分析自動化工具")
    gr.Markdown("此工具會執行：**PTT 爬蟲 → 寫入 Sheet → TF-IDF 關鍵詞分析 → 寫入統計表 → Gemini AI 洞察生成**")

    with gr.Row():
        gr.Markdown(f"**Google Sheet URL:** [分析試算表連結]({SHEET_URL})")

    with gr.Tab("一鍵分析執行區"):
        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### 爬蟲參數 (Crawl Settings)")
                index_pages = gr.Number(label="往前翻頁數 (Pages)", value=3, precision=0)
                min_push = gr.Number(label="最小推文數 (Min Nrec)", value=10, precision=0)
                keyword = gr.Textbox(label="標題關鍵詞篩選 (Keyword)", value="", placeholder="可選，例如：阿凡達")

                gr.Markdown("### 文本分析參數 (Analysis Settings)")
                topk = gr.Number(label="輸出 Top K 關鍵詞", value=50, precision=0)
                min_df = gr.Number(label="最小文件頻率 (Min DF)", value=2, precision=0, info="詞彙至少要在幾篇文章中出現")

                btn_run = gr.Button("🚀 一鍵執行完整流程並生成報告", variant="primary")

            with gr.Column(scale=2):
                gr.Markdown("### 🤖 分析報告 (Analysis Report)")
                report_output = gr.Markdown("點擊按鈕開始分析...")

        gr.Markdown("---")
        gr.Markdown("### 💾 關鍵詞統計結果預覽 (Top Terms Preview)")
        grid_terms = gr.Dataframe(value=terms_df, label="關鍵詞統計數據 (已寫入 Sheet)", interactive=False)

    with gr.Tab("原始貼文預覽"):
        gr.Markdown("### 原始貼文數據 (已累積在 Sheet 中)")
        grid_posts = gr.Dataframe(value=ptt_posts_df, label="PTT 文章資料", interactive=False)
        btn_refresh_data = gr.Button("⬇️ 從 Sheet 讀取最新數據")

    # --- 綁定動作 ---

    # 執行流程
    btn_run.click(
        full_analysis_workflow,
        inputs=[index_pages, min_push, keyword, topk, min_df],
        outputs=[report_output, grid_terms]
    )

    # 重新讀取數據
    btn_refresh_data.click(
        _refresh_data,
        outputs=[grid_posts, grid_terms]
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://247a988d55b4f26592.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
